<a href="https://colab.research.google.com/github/kimhee96/NLP/blob/master/business_goal_tfidf_0803.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import pickle

In [19]:
#Data Load

data_path = os.path.join(os.getcwd(), 'drive','My Drive','business_goal')
tr = pd.read_csv(os.path.join(data_path,'train.csv'))
te = pd.read_csv(os.path.join(data_path,'test.csv'))

In [26]:
tr.head()

,X,regist_num,class_code,business_goal,Description,Category
0,68,668,1101111847031,A01110,농 수 축산물 제조 가공 및 판매업 단체급식 및 써비스업 캐터링사업잔체급식 사업 농...,A
1,2758,38477,1101112188913,A01110,영농업 골재 채취업 도 소매 써비스업 군납업 기동단 공원묘지 및 공원 단지 조성업 ...,A
2,3984,51621,1101112332635,A01110,농산물 재배 가공 제조 및 유통 판매업 농산물 수입 판매업무역 비료 생산 판매업 ...,A
3,9282,87208,1101113011452,A01110,농업의경영 관광농원 및 주말농장 설치운영 집단재배 및 공동작업에 관한 사항 농업에 ...,A
4,19041,161987,1101114182997,A01110,농 수 축산물 생산 농 수 축산물 도 소매업 농 수 축산물 가공 농 수 축산물 수출...,A


In [27]:
te.head()

,X,regist_num,class_code,business_goal,category,Unnamed: 5
0,74201,666824,1612710003908,A01110,집단재배 및 공동작업에 관한 사업 농 축산업에 관련된 공동이용시설의 설치 및 운영 ...,A
1,74373,668418,1613710005366,A01110,농업경영 농산물 유통사업 및 농산물 수입 수출업 관광농업 농산물 계약 재배 농산물 ...,A
2,74374,668433,1613710005481,A01110,농업의 경영 및 부대사업 농업에 관련된 공동이용시설의 설치 운영 농산물의 공동출하 ...,A
3,74545,669878,1614710009879,A01110,친환경 농 축산물의 생산기술 보급과 교류 농 축 특산물 임산물 유통에 관한 조사연구...,A
4,74546,669891,1614710009952,A01110,집단재배 및 공동작업에 관한 사업 농업에 관련된 공동이용시설의 설치 및 운영 농기계...,A


In [23]:
# GPU 상태확인
!nvidia-smi

Sun Aug  2 11:27:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        711M        9.7G        968K        2.3G         11G
Swap:            0B          0B          0B


In [25]:
# train 전체를 사용하면 램 용량때문에 session down이 일어난다. 
# 따라서 test데이터만 사용하여 성능을 확인한다. 
# test데이터의 70%를 train / 나머지를 test로 새로운 데이터를 만든다.  
sentences = te['category']
#양 끝 공백 제거
tr_sentences = [str(sentence.strip()) for sentence in sentences]

In [28]:
vect = TfidfVectorizer()
X = vect.fit_transform(tr_sentences)
X.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
labels = list(te['Unnamed: 5'])
uniq_labels = sorted(list(set(labels)))
labels_dict = {}
for i, e in enumerate(uniq_labels) :
  labels_dict[e] = i
tr_labels = list((pd.Series(labels)).map(labels_dict))

In [30]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(X,
                                                                                    tr_labels, 
                                                                                    random_state=2020, 
                                                                                  test_size=0.3)


In [31]:
# 모델 fitting
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()
model.fit(train_inputs, train_labels)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [32]:
#RAM 용량 한계로 학습 모델 저장
modelname = 'sgd_base_model.sav'
pickle.dump(model, open(modelname, 'wb'))


In [33]:
#model load

modelname = 'sgd_base_model.sav'
model = pickle.load(open(modelname,'rb'))

y_pred = model.predict(validation_inputs)

In [34]:
y_pred.shape

(6374,)

In [36]:
len(validation_labels)

6374

In [37]:
from sklearn.metrics import accuracy_score

score = accuracy_score(validation_labels, y_pred)

In [38]:
score

0.6545340445560088